In [6]:
import os
import warnings
import hashlib
from dotenv import load_dotenv
from supabase import create_client, Client
import numpy as np
from tqdm import tqdm
import pandas as pd
import sys
sys.path.append('../scripts')
from get_weather import fetch_weather_for_all_stations

warnings.filterwarnings("ignore")
load_dotenv()

True

In [7]:
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [8]:
def get_all_data_paginated(table_name: str, page_size: int = 9999):
        all_data = []
        offset = 0
        while True:
            response = supabase.from_(table_name).select("*").range(offset, offset + page_size - 1).execute()
            data = response.data
            if not data:
                break
            all_data.extend(data)
            offset += page_size
        return all_data

In [9]:
trips = get_all_data_paginated("trips")
df = pd.DataFrame(trips)

In [10]:
df.sort_values(by=['date', 'train_id', 'sequence'], inplace=True)
subtrip_rows = []
for train_id, group in tqdm(df.groupby('train_id')):
    for i in range(len(group) - 1):
        row = group.iloc[i]
        next_row = group.iloc[i + 1]
        if next_row['sequence'] != row['sequence'] + 1:
            continue
        subtrip = row.copy()

        subtrip['next_station_name'] = next_row['station_name']

        # Adjust the departure and arrival times for the subtrip
        subtrip['scheduled_departure_time'] = row['scheduled_departure_time']
        subtrip['actual_departure_time'] = row['actual_departure_time']
        subtrip['scheduled_arrival_time'] = next_row['scheduled_arrival_time']
        subtrip['actual_arrival_time'] = next_row['actual_arrival_time']
        subtrip['sequence'] = row['sequence']
        subtrip['departure_delay'] = row['departure_delay']
        subtrip['arrival_delay'] = next_row['arrival_delay']

        # Append the new subtrip row
        subtrip_rows.append(subtrip)
df = pd.DataFrame(subtrip_rows)

100%|██████████| 239/239 [02:55<00:00,  1.36it/s]


In [11]:
# rename columns for clarity
df.rename(columns={
    'id': 'subtrip_id',
    'station_name': 'current_station',
    'next_station_name': 'next_station'}, inplace=True)

# Convert date and time columns to appropriate formats
df['date'] = pd.to_datetime(df['date']).dt.date
df['scheduled_arrival_time'] = pd.to_datetime(df['scheduled_arrival_time']).dt.time
df['scheduled_departure_time'] = pd.to_datetime(df['scheduled_departure_time']).dt.time
df['actual_arrival_time'] = pd.to_datetime(df['actual_arrival_time']).dt.time
df['actual_departure_time'] = pd.to_datetime(df['actual_departure_time']).dt.time
df['arrival_hour'] = pd.to_datetime(df['scheduled_arrival_time'].astype(str), errors='coerce').dt.hour
df['departure_hour'] = pd.to_datetime(df['scheduled_departure_time'].astype(str), errors='coerce').dt.hour


# add additional columns
df['trip_id'] = df.apply(lambda row: hashlib.sha256(f"{row['date']}_{row['initial_departure_station']}_{row['final_arrival_station']}_{row['train_id']}".encode()).hexdigest(), axis=1)
df['route'] = df['initial_departure_station'] + ' - ' + df['final_arrival_station']
df['day_of_week'] = pd.to_datetime(df['date']).dt.weekday # Monday=0, Sunday=6
df['number_of_stations'] = df.groupby('trip_id')['current_station'].transform('nunique') + 1 # +1 for the final station

# convert scheduled arrival and departure times to datetime, using date column
df['scheduled_arrival_time'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['scheduled_arrival_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
df['scheduled_departure_time'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['scheduled_departure_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
df['scheduled_arrival_time_diff'] = df['scheduled_arrival_time'] - df.groupby('trip_id')['scheduled_departure_time'].transform('first')
df['scheduled_departure_time_diff'] = df['scheduled_departure_time'] - df.groupby('trip_id')['scheduled_departure_time'].transform('first')

# Vectorized adjustment of scheduled times to avoid negative differences
mask_arrival = df['scheduled_arrival_time_diff'] < pd.Timedelta(0)
mask_departure = df['scheduled_departure_time_diff'] < pd.Timedelta(0)
df.loc[mask_arrival, 'scheduled_arrival_time'] += pd.Timedelta(days=1)
df.loc[mask_departure, 'scheduled_departure_time'] += pd.Timedelta(days=1)

# Add trip duration in minutes
first_departure = df.groupby('trip_id')['scheduled_departure_time'].min()
last_arrival = df.groupby('trip_id')['scheduled_arrival_time'].max()
trip_duration = (last_arrival - first_departure).dt.total_seconds() / 60  # in minutes
df['trip_duration'] = df['trip_id'].map(trip_duration)

df['subtrip_duration'] = df.apply(lambda row: pd.to_datetime(row['scheduled_arrival_time'].strftime('%H:%M:%S'), format='%H:%M:%S') - pd.to_datetime(row['scheduled_departure_time'].strftime('%H:%M:%S'), format='%H:%M:%S'), axis=1)
df['subtrip_duration'] = df['subtrip_duration'].dt.total_seconds() / 60

# calculate distance in km for each trip and subtrip
# with open('../data/distances.csv', 'r') as f:
#     distances = pd.read_csv(f)

# distances_indexed = distances.set_index(['initial_departure_station', 'final_arrival_station', 'train_type'])['distance']
# df['trip_distance'] = df.set_index(['initial_departure_station', 'final_arrival_station', 'train_type']).index.map(distances_indexed.get)

# Calculate subtrip distance
with open('../data/subtrip_distances_new.csv', 'r') as f:
    subtrip_distances = pd.read_csv(f)

subtrip_distances_indexed = subtrip_distances.set_index(['departure_station', 'arrival_station', 'train_type'])['new_distance']
df['subtrip_distance'] = df.set_index(['current_station', 'next_station', 'train_type']).index.map(subtrip_distances_indexed.get)
# round subtrip distance to 2 decimal places
df['subtrip_distance'] = df['subtrip_distance'].round(2)

# use subtrip distance to calculate trip distance
df['trip_distance'] = df.groupby('trip_id')['subtrip_distance'].transform('sum')

In [12]:
# add dwelling time
df['scheduled_dwelling_time'] = df['scheduled_departure_time'] - df.groupby('trip_id')['scheduled_arrival_time'].shift(1)
df['scheduled_dwelling_time'] = df['scheduled_dwelling_time'].dt.total_seconds() // 60  # convert to minutes
df['scheduled_dwelling_time'].fillna(0, inplace=True)

# add travelled and remaining distance
df['remaining_distance'] = df['trip_distance'] - df.groupby('trip_id')['subtrip_distance'].cumsum() + df['subtrip_distance']
df['remaining_distance'] = df['remaining_distance'].round(2)
df['travelled_distance'] = df.groupby('trip_id')['subtrip_distance'].cumsum() - df['subtrip_distance']
df['travelled_distance'] = df['travelled_distance'].round(2)

In [13]:
df = df[['trip_id', 'subtrip_id', 'date', 'day_of_week', 'train_id', 'train_type', 'initial_departure_station', 'final_arrival_station', 'route', 'current_station', 'next_station', 'sequence', 'number_of_stations', 'trip_duration', 'subtrip_duration', 'trip_distance', 'subtrip_distance', 'travelled_distance', 'remaining_distance', 'scheduled_departure_time', 'scheduled_arrival_time', 'actual_departure_time', 'actual_arrival_time',  'departure_delay', 'arrival_delay', 'departure_hour', 'arrival_hour', 'scheduled_dwelling_time']]

In [14]:
# get min and max dates


min_date = df['date'].min()
max_date = df['date'].max()
weather = fetch_weather_for_all_stations(start_date=min_date, end_date=max_date)

# weather = pd.read_csv('../data/weather_data.csv') # or this
weather

  0%|          | 0/97 [00:00<?, ?it/s]

100%|██████████| 97/97 [00:55<00:00,  1.75it/s]


Weather data saved to 'weather_data.csv' with 195552 records.


,date,latitude,longitude,station_name,timezone,timezone_abbreviation,temperature,relative_humidity,dew_point,apparent_temperature,precipitation,visibility,wind_speed,wind_direction,wind_gusts,uv_index,cloud_cover,surface_pressure
0,2025-05-18 00:00:00+01:00,31.6250,-8.0,MARRAKECH,b'Africa/Casablanca',b'GMT+1',19.371500,66.0,12.857771,19.605425,0.0,56440.0,4.582052,224.999893,10.799999,0.00,3.0,962.684204
1,2025-05-18 01:00:00+01:00,31.6250,-8.0,MARRAKECH,b'Africa/Casablanca',b'GMT+1',18.621500,69.0,12.822044,18.805630,0.0,50920.0,4.843305,228.012863,11.520000,0.00,0.0,962.080750
2,2025-05-18 02:00:00+01:00,31.6250,-8.0,MARRAKECH,b'Africa/Casablanca',b'GMT+1',17.771502,72.0,12.657198,17.877752,0.0,47080.0,5.001280,239.743652,12.599999,0.00,20.0,961.743774
3,2025-05-18 03:00:00+01:00,31.6250,-8.0,MARRAKECH,b'Africa/Casablanca',b'GMT+1',17.321501,72.0,12.224657,17.416338,0.0,46220.0,4.104631,232.124954,11.879999,0.00,8.0,961.286072
4,2025-05-18 04:00:00+01:00,31.6250,-8.0,MARRAKECH,b'Africa/Casablanca',b'GMT+1',16.971500,73.0,12.097652,17.088531,0.0,45400.0,3.671294,258.690094,9.720000,0.00,0.0,961.130188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195547,2025-08-09 19:00:00+01:00,34.5625,-3.0,MELG EL OUIDANE,b'Africa/Casablanca',b'GMT+1',30.608500,51.0,19.322868,30.546127,0.0,70360.0,24.569832,10.980613,44.279999,1.55,0.0,988.524597
195548,2025-08-09 20:00:00+01:00,34.5625,-3.0,MELG EL OUIDANE,b'Africa/Casablanca',b'GMT+1',29.108500,53.0,18.556808,29.352383,0.0,69780.0,19.995399,13.535882,38.880001,0.35,0.0,988.976868
195549,2025-08-09 21:00:00+01:00,34.5625,-3.0,MELG EL OUIDANE,b'Africa/Casablanca',b'GMT+1',27.708500,58.0,18.696873,28.704834,0.0,65840.0,15.294703,26.564985,30.960001,0.00,0.0,989.631226
195550,2025-08-09 22:00:00+01:00,34.5625,-3.0,MELG EL OUIDANE,b'Africa/Casablanca',b'GMT+1',27.158501,57.0,17.906086,28.014698,0.0,67080.0,13.779114,19.855309,23.759998,0.00,0.0,990.847046


In [15]:
weather['hour'] = pd.to_datetime(weather['date']).dt.hour
weather['date'] = pd.to_datetime(weather['date']).dt.date
weather.drop(columns=['latitude', 'longitude', 'timezone', 'timezone_abbreviation'], inplace=True)
weather.columns

Index(['date', 'station_name', 'temperature', 'relative_humidity', 'dew_point',
       'apparent_temperature', 'precipitation', 'visibility', 'wind_speed',
       'wind_direction', 'wind_gusts', 'uv_index', 'cloud_cover',
       'surface_pressure', 'hour'],
      dtype='object')

In [16]:
df = df.merge(weather.add_suffix('_on_departure'), left_on=['date', 'departure_hour', 'current_station'], right_on=['date_on_departure', 'hour_on_departure', 'station_name_on_departure'], how='left')
df = df.merge(weather.add_suffix('_on_arrival'), left_on=['date', 'arrival_hour', 'next_station'], right_on=['date_on_arrival', 'hour_on_arrival', 'station_name_on_arrival'], how='left')
df.drop(columns=['date_on_departure', 'hour_on_departure', 'station_name_on_departure', 'date_on_arrival', 'hour_on_arrival', 'station_name_on_arrival'], inplace=True)

In [17]:
df[:150]

,trip_id,subtrip_id,date,day_of_week,train_id,train_type,initial_departure_station,final_arrival_station,route,current_station,...,dew_point_on_arrival,apparent_temperature_on_arrival,precipitation_on_arrival,visibility_on_arrival,wind_speed_on_arrival,wind_direction_on_arrival,wind_gusts_on_arrival,uv_index_on_arrival,cloud_cover_on_arrival,surface_pressure_on_arrival
0,475a318f125109daa63f2d22bbc975d685afbc3e2aadb5...,9ecce6061fb759c37f51dcd0578a3514724cb3d7d3da19...,2025-05-19,0,1,TNR,CASA PORT,KENITRA,CASA PORT - KENITRA,CASA PORT,...,14.521266,17.952110,0.0,23160.0,5.411986,93.814003,14.040000,0.00,62.0,1014.913086
1,475a318f125109daa63f2d22bbc975d685afbc3e2aadb5...,c457d66f9f93dff1bcb117f0c6b78947f552efd2634588...,2025-05-19,0,1,TNR,CASA PORT,KENITRA,CASA PORT - KENITRA,AIN SEBAA,...,14.502754,17.370037,0.0,17040.0,4.198285,120.963692,11.159999,0.00,62.0,1017.299683
2,475a318f125109daa63f2d22bbc975d685afbc3e2aadb5...,4801ea335f904b86b6ae8e7afeeaf483df6e521145cd08...,2025-05-19,0,1,TNR,CASA PORT,KENITRA,CASA PORT - KENITRA,MOHAMMEDIA,...,14.242029,17.002121,0.0,19620.0,6.725354,105.524178,11.879999,0.00,60.0,1013.948792
3,475a318f125109daa63f2d22bbc975d685afbc3e2aadb5...,098075ef8b0b1a9171cd5df6955895561f5dce9689fd02...,2025-05-19,0,1,TNR,CASA PORT,KENITRA,CASA PORT - KENITRA,BOUZNIKA,...,13.803034,16.893068,0.0,21980.0,4.582052,135.000107,10.799999,0.05,64.0,1014.183228
4,475a318f125109daa63f2d22bbc975d685afbc3e2aadb5...,08eec44849939fe9c3e0951a65d788facd5c8f0f672bd1...,2025-05-19,0,1,TNR,CASA PORT,KENITRA,CASA PORT - KENITRA,SKHIRAT,...,13.732921,16.796690,0.0,21980.0,4.582052,135.000107,10.799999,0.05,64.0,1012.867310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,18c0733d1bd1fdf2235c3a6d56b35d323cfd3f597388e0...,e5bf71f201b9b5fe84cf1d2a26098c20a1a9fb429a2e01...,2025-06-04,2,1,TNR,CASA PORT,KENITRA,CASA PORT - KENITRA,TEMARA,...,12.878490,18.526283,0.0,50540.0,1.138420,288.435028,1.080000,0.05,54.0,1016.191589
146,18c0733d1bd1fdf2235c3a6d56b35d323cfd3f597388e0...,45d28269ae003d7528cfd0463e9ba0dce080d5ac5a9070...,2025-06-04,2,1,TNR,CASA PORT,KENITRA,CASA PORT - KENITRA,RABAT AGDAL,...,13.073643,18.315660,0.0,44300.0,1.484318,165.963730,1.440000,0.05,58.0,1015.931519
147,18c0733d1bd1fdf2235c3a6d56b35d323cfd3f597388e0...,3852dfa7629d2828a9bcf09a8df1d0f5c700d8a3e07c69...,2025-06-04,2,1,TNR,CASA PORT,KENITRA,CASA PORT - KENITRA,RABAT VILLE,...,12.891006,18.543505,0.0,50540.0,1.138420,288.435028,1.080000,0.05,54.0,1016.430176
148,18c0733d1bd1fdf2235c3a6d56b35d323cfd3f597388e0...,45b6b64a2a124daa8548b986a887a719ca090430bb1e72...,2025-06-04,2,1,TNR,CASA PORT,KENITRA,CASA PORT - KENITRA,SALE,...,12.878490,18.526283,0.0,50540.0,1.138420,288.435028,1.080000,0.05,54.0,1016.191589


In [18]:
df.to_csv('../data/trips_data.csv', index=False)